In [55]:
import numpy as np
from metavision_core.event_io import EventsIterator, RawReader
from metavision_sdk_core import RoiFilterAlgorithm, FlipXAlgorithm, RollingEventCDBuffer, RollingEventBufferConfig, BaseFrameGenerationAlgorithm
from metavision_sdk_cv import RotateEventsAlgorithm, SpatioTemporalContrastAlgorithm, AntiFlickerAlgorithm
from metavision_sdk_analytics import TrackingAlgorithm, TrackingConfig, draw_tracking_results
from math import pi
from matplotlib import pyplot as plt

################
## Parameters ##
################
input_path = r"D:\CZI_scope\code\data\raw\recording_2024-07-30_13-51-19.raw"
width = 640
height = 480
acc_time = 10  # in us
start_time = 2.6e7
duration = 3e6  # in us
min_size_um = 1
max_size_um = 6
mag = 38
cmos_pitch = 3.45
ebis_pitch = 15
update_frequency = 20 # in Hz

#############
## Helpers ##
#############
def start_ts():
    ts = int((start_time // acc_time) * acc_time)
    return (ts - (ts % acc_time))

def um_to_pix(pitch, mag, um):
    return int(um / pitch * mag)


def pix_to_um(pitch, mag, pix):
    return int(pix * pitch / mag)

def update_period():
    T = 1e6 / update_frequency
    return int(T // acc_time) * acc_time


#####################
## Algorithm setup ##
#####################
stc_filter = SpatioTemporalContrastAlgorithm(width, height, 1000, True)
anti_flicker_filter = AntiFlickerAlgorithm(width, height)
x1, y1, x2, y2 = (265, 100, 430, 400)
roi_filter = RoiFilterAlgorithm(
    x1, y1, x2, y2, output_relative_coordinates=False)
rot_algo = RotateEventsAlgorithm(height - 1, width - 1, -pi/2)
flip_x_algo = FlipXAlgorithm(width - 1)


# Tracking Algorithm
tracking_config = TrackingConfig()  # Default configuration
tracking_config.tracker = TrackingConfig.Tracker.Ellipse
tracking_config.ellipse_tracker_update_function = TrackingConfig.EllipseUpdateFunction.Gaussian
tracking_config.ellipse_tracker_update_method = TrackingConfig.EllipseUpdateMethod.PerEvent
tracking_algo = TrackingAlgorithm(
    sensor_width=width, sensor_height=height, tracking_config=tracking_config)
tracking_algo.min_size = um_to_pix(ebis_pitch, mag, min_size_um)
tracking_algo.max_size = um_to_pix(ebis_pitch, mag, max_size_um)

#############
## Buffers ##
#############
tracking_buf = tracking_algo.get_empty_output_buffer()
rot_buf = rot_algo.get_empty_output_buffer()
flip_buf = flip_x_algo.get_empty_output_buffer()
roi_buf = roi_filter.get_empty_output_buffer()                        
stc_buf = stc_filter.get_empty_output_buffer()

In [58]:
buffer_config = RollingEventBufferConfig.make_n_us(acc_time)
buffer = RollingEventCDBuffer(buffer_config)
mv_iterator = EventsIterator(input_path=input_path, start_ts=start_ts(),
                             max_duration=int(duration),
                             delta_t=update_period(), mode="delta_t")

tracking_results = []


def filter_events(events):
    anti_flicker_buf = anti_flicker_filter.get_empty_output_buffer()
    rot_algo.process_events(events, rot_buf)
    flip_x_algo.process_events(rot_buf, flip_buf)
    roi_filter.process_events(flip_buf, roi_buf)
    anti_flicker_filter.process_events(roi_buf, anti_flicker_buf)
    return anti_flicker_buf

def process_tracking(events):
    buffer.insert_events(events)
    tracking_algo.process_events(buffer, tracking_buf)

for events in mv_iterator:
    events = filter_events(events)
    process_tracking(events)

print(f"Number of events: {tracking_buf.numpy().size}")

Number of events: 0
